<a href="https://colab.research.google.com/github/GauraoM/Tensorflow/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

### Dataset
#### 3 classes of species Setosa, Versicolor, Verginica
#### Information: sepal length, sepal width, petal length, petal width  

In [ ]:
# set some constants
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
# Get the file and save it as their name given
train_path = tf.keras.utils.get_file("iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

# reading the data
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header=0)

In [ ]:
train.head() # returns 5 rows 

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
# pop the species column so that can be used as label
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


#### Input Function

In [40]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle and repeat if in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

#### Feature columns

In [41]:
# It describes how to use input
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


#### Model building

In [42]:
# Build a DNN classifier with two hidden layers having 30,10 nodes and numberr of classes are 3
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30, 10],n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpegzh1umk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### Training the Model 

In [43]:
# Includes lambda to avoid creating input function
classifier.train(input_fn = lambda: input_fn(train, train_y, training=True),steps=5000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpegzh1umk/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.9777102, step = 0
INFO:tensorflow:global_step/sec: 348.149
INFO:tensorflow:loss = 1.0925208, step = 100 (0.293 sec)
INFO:tensorflow:global_step/sec: 410.604
INFO:tensorflow:loss = 0.9771913, step = 200 (0.241 sec)
INFO:tensorflow:global_step/sec: 393.595
INFO:tensorflow:loss = 0.9281351, step = 300 (0.257 sec)
INFO:tensorflow:global_step/sec: 369.855
INFO:tensorflow:loss = 0.89862204, step = 4

In [44]:
# Evaluation
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-03-21T06:23:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpegzh1umk/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36135s
INFO:tensorflow:Finished evaluation at 2022-03-21-06:23:11
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.81666666, average_loss = 0.506233, global_step = 5000, loss = 0.506233
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpegzh1umk/model.ckpt-5000

Test set accuracy: 0.817



#### Predictions


In [49]:
def input_fn(features, batch_size=25):
  # Converting the inputs to a Dataset without labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
# Iterates through features
for feature in features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit(): # if not a digit
      valid = False
  predict[feature] = [float(val)] 


# perform prediction
predictions = classifier.predict(input_fn=lambda: input_fn(predict))
# Iterating over prediction
for pred_dict in predictions:
    print(pred_dict)
  # Get the class id
    class_id = pred_dict['class_ids'][0]
    # Get the probability
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
SepalLength:6.4
SepalWidth:6.5
PetalLength:5.3
PetalWidth:5.6
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpegzh1umk/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([2.735777 , 2.9611075, 3.082922 ], dtype=float32), 'probabilities': array([0.27264607, 0.34155378, 0.38580018], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (38.6%)
